In [ ]:
!pip install profanity-check==1.0.3

In [ ]:
!pip install pyarrow

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import pyarrow as pa   
import pyarrow.parquet as pq
#from fastparquet import ParquetFile
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob, Word
from sklearn.externals import joblib
import json

In [ ]:
from profanity_check import predict, predict_prob

predict(['predict() takes an array and returns a 1 for each string if it is offensive, else 0.'])
# [0]

predict(['fuck you'])
# [1]

predict_prob(['predict_prob() takes an array and returns the probability each string is offensive'])
# [0.08686173]

predict_prob(['go to hell, you scum'])
# [0.7618861]

In [ ]:
translations = translator.translate(['The quick brown fox', 'jumps over', 'the lazy dog'], dest='ko')
for translation in translations:
    print(translation.origin, ' -> ', translation.text)

In [ ]:
! pip install googletrans

In [ ]:
import cx_Oracle
import pandas as pd 

In [ ]:
con =  cx_Oracle.connect('comprasweb_ij/Xxxx1234!@10.10.2.223/MKTG')
print(con.version)

df = pd.read_sql_query('''
SELECT * 
FROM --DISTINCT para empresas
(
SELECT 
	RATINGEVENTOBJECTTARGETID,
	s.DT,
	TRUNC(s.RATINGEVENTOBJECTINSTANT) AS fecha_review,
	s.RATINGEVENTOBJECTSCORE,
	REPLACE(REPLACE(s.REVIEWNAME, '?', 'n'),'n', 'n') AS REVIEWNAME,
    REPLACE(REPLACE(s.REVIEWVALUE, '?', 'n'),'n', 'n') AS REVIEWVALUE
FROM DWHPRO.COMPANY_REVIEW s 
WHERE --to_date(DT) BETWEEN to_date('01/09/2018') AND to_date('31/12/2019')
s.REVIEWVALUE like '%mierda%'
)t 
pivot (
min(REVIEWVALUE) value FOR REVIEWNAME IN ('title', 'description')
)
''', con)
df_save = df

con.close()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df[['RATINGEVENTOBJECTTARGETID','DT','RATINGEVENTOBJECTSCORE',"'title'_VALUE","'description'_VALUE"]]

In [ ]:
df = df.rename(columns={"RATINGEVENTOBJECTTARGETID": "EXPERIENCE_ID"
                            ,"DT": "FECHA"
                            ,"RATINGEVENTOBJECTSCORE": "RATING"
                            ,"'title'_VALUE": "TITLE"
                            ,"'description'_VALUE": "DESCRIPTION"
                           })

In [ ]:
#df['T+D'] = df[['TITLE', 'DESCRIPTION']].apply(lambda x: ''.join(x), axis=1)
df['T+D'] = df.TITLE.astype(str).str.cat(df.DESCRIPTION.astype(str), sep=' ')

In [ ]:
df["is_bad_review"] = df["RATING"].apply(lambda x: 1 if x < 3 else 0)
# select only relevant columns
df.head()

In [ ]:
#quitar ultimos caracteres extraños manualmente
df["T+D"] = df["T+D"].apply(lambda x: 
x.replace("!", "").replace("?", "n").replace("Enter", "").replace("enter", "").replace("None", "")
)

In [ ]:
#df_bad = df[df['T+D'].notnull()]
df_bad = df[df['T+D'].notnull()]

In [ ]:
df_bad.head(5)

In [ ]:
#df_bad = df_bad[df_bad["is_bad_review"]==1]

In [ ]:
#df_bad = df_bad[df_bad['DESCRIPTION'].notnull()]

In [ ]:
#df_bad = df_bad[df_bad['TITLE'].notnull()]

In [ ]:
#df_bad = df_bad.head(5)

In [ ]:
import string
punctuation=string.punctuation

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words('spanish')

In [ ]:
df_example = df_bad

In [ ]:
df_example.head()

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [ ]:
def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stop and tok not in punctuation]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)

In [ ]:
df_example['clean_text'] = df_example['T+D'].apply(lambda x: cleanup_text(x, False))

In [ ]:
#df_example=df_example.head(10)

In [ ]:
#df_bad['T+D'] = df_bad['T+D'].astype(str)

In [ ]:
from googletrans import Translator
translator = Translator()
df_example['English'] = df_example["DESCRIPTION"].map(lambda x: translator.translate(x, src="es", dest="en").text)

In [ ]:
df_example['prob'] = predict_prob(df_example['English'])

In [ ]:
df_example['prob_es'] = predict_prob(df_example['example'])

In [ ]:
df_example['prof'] = predict(df_example['English'])

In [ ]:
df_example.head()

In [ ]:
! pip install -U textblob

In [ ]:
! python -m textblob.download_corpora

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
x = df_example['RATING'].value_counts () 
x = x.sort_index () 
plt.figure(figsize = (10,6)) 
ax = sns.barplot (x.index, x.values, alpha = 0.8) 
plt.title ("Star Rating Distribution") 
plt.ylabel ('count') 
plt.xlabel ('Star Ratings') 
rects = ax.patches 
labels = x.values 
for rect, label in zip (rects, labels): 
    height = rect.get_height () 
    ax.text (rect.get_x () + rect.get_width () / 2, height + 5, label, ha = 'center', va = 'bottom') 
plt.show ();

In [ ]:
#sample_reviews = df_example[['RATING','T+D']].sample(1000000) 
def detect_polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
df_example['polarity'] = df_example.clean_text.apply(detect_polarity)
df_example

In [ ]:
#sample_reviews = df_example[['RATING','T+D']].sample(1000000) 
def translate_text(text):
    return TextBlob(text).translate(to="en")

In [ ]:
df_example['ingles'] = df_example.clean_text.apply(translate_text)
df_example

In [ ]:
df_example['english'] = df_example['clean_text'].apply(lambda x:TextBlob(x.strip()).translate(to='en'))

In [ ]:
pol = lambda x: TextBlob(x).sentiment.polarity
pol2 = lambda x: x.sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
sub2 = lambda x: x.sentiment.subjectivity
 
traducir = lambda x: TextBlob(x).translate(to="en")
 
df_example['blob_en'] = df_example['clean_text'].apply(traducir)
df_example['polarity'] = df_example['blob_en'].apply(pol2)
df_example['subjectivity'] = df_example['blob_en'].apply(sub2)
df_example

In [ ]:
! pip install py-translate

In [ ]:
from translate import translator